In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
  0% 0.00/39.9k [00:00<?, ?B/s]
100% 39.9k/39.9k [00:00<00:00, 32.2MB/s]
Archive:  poem-generation.zip
  inflating: data/poem.txt           


In [ ]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/.credentials/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download harshalgadhe/poem-generation
!unzip -d data/ poem-generation.zip 